We use arabic tweet dataset

In [ ]:
!wget https://raw.githubusercontent.com/mahmoudnabil/ASTD/master/data/Tweets.txt

--2022-04-23 20:14:08--  https://raw.githubusercontent.com/mahmoudnabil/ASTD/master/data/Tweets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1561622 (1.5M) [text/plain]
Saving to: ‘Tweets.txt’

Tweets.txt          100%[===================>]   1.49M  --.-KB/s    in 0.06s   

2022-04-23 20:14:09 (26.9 MB/s) - ‘Tweets.txt’ saved [1561622/1561622]



## Imports

In [ ]:
import pandas as pd # for reading csv file
import tensorflow as tf
import re
import numpy as np
import csv 
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from sklearn.utils import shuffle
from nltk.corpus import stopwords # list of stop words
import nltk # for processing texts
from io import StringIO
nltk.download('stopwords')
from sklearn.model_selection import train_test_split  # for splitting data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
read_file = pd.read_csv (r'Tweets.txt',on_bad_lines='skip')#
read_file.to_csv (r'DatasetTweets.csv', index=None)
data = pd.read_csv(r"DatasetTweets.csv")


In [ ]:
data.head()

,text,polarity
0,بعد استقالة رئيس #المحكمة_الدستورية ننتظر استق...,Neutre
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,Positive
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,Negative
3,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,Neutre
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,Neutre


## Read the Dataset

preprocess a review by removing special characters and long spaces (clean dataset)

In [ ]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' # define arabic punctuations

def process_review(review):
  out = re.sub(r"[^\w\s]", '', review)
  out = re.sub(r"[a-zA-Z]", '', out)# remove english letters
  out = re.sub(r"\n", '', out) # remove \n from text
  out = re.sub(r'\d+', '', out) #remove number
  out = re.sub(r'http\S+', '', out) # remove links
  out = re.sub(r"["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]", '', out)                    #remove emojis
  out = out.translate(str.maketrans('','', arabic_punctuations)) # remove punctuation
  out = ' '.join([word for word in out.split() if word not in stopwords.words("arabic")]) # remove stop word
  out = re.sub(r"\s+", ' ', out)
  out = re.sub(' +', ' ',out) # remove extra space
  return out.strip()

In [ ]:
# count of each type 

data['polarity'].value_counts()


Negative     7946
Neutre       7267
Positive     7151
sentiment       1
Name: polarity, dtype: int64

In [ ]:
#befor cleaning
data['text'][433]

'مين جاي مؤتمر بكره؟ ": مؤتمر الدفاع عن الدستور يستعرض المؤتمر آخر مستجدات وتطورات عملية وضع الدستور وتبعاتها "\t'

In [ ]:
# The cleaning function applied in all texts
data['text'] = data['text'].apply(process_review)

In [ ]:
#after cleaning
data['text'][433]

'مين جاي مؤتمر بكره مؤتمر الدفاع الدستور يستعرض المؤتمر آخر مستجدات وتطورات عملية وضع الدستور وتبعاتها'

In [ ]:
    
#shuffle the data

print(len(data['text']))

22366


Look at the data

In [ ]:
for i in range(0, 10):
  print(data)

                                                    text  polarity
0      استقالة رئيس المحكمةالدستورية ننتظر استقالة رئ...    Neutre
1      أهنئ الدكتور أحمد جمال الدين القيادي بحزب مصر ...  Positive
2      البرادعي يستقوى بامريكا مرةاخرى يرسل عصام العر...  Negative
3      الحريةوالعدالة شاهد ليلةالاتحادية فيلم استقصائ...    Neutre
4      الوالدة اقولها بخاطري حشيشة تضحك اقولها ملل ال...    Neutre
...                                                  ...       ...
22361  ربي اغفر لوالدي لأحبتي للمؤمنين المؤمنات المسل...  Positive
22362                                   ربي يسعدنا وياكم  Positive
22363  يتحدثون اخلاق حسين ونجوم فرقهم نهاياتهم الرياض...  Positive
22364  صباحكم احتفالية تكتمل وصاحب الاحتفاليه ماكمل ا...  Positive
22365               قلت سابقا المعيوف عندما تحتاجه يخذلك  Positive

[22366 rows x 2 columns]
                                                    text  polarity
0      استقالة رئيس المحكمةالدستورية ننتظر استقالة رئ...    Neutre
1      أهنئ الدكتور أحمد جمال الدين 

In [ ]:
#save the dataset
text=data['text']
sentiment=data['polarity']
with open('data.txt', 'w') as f:
  for i in range(len(data['text'])):
    f.write(f"{text[i]}, {sentiment[i]} '\n'")

## Create Sequences
Create sequences by using the most repeated 500 words

In [ ]:
tknzr = Tokenizer(lower=True, split=" ")
tknzr.fit_on_texts(data['text']) #maj le vocabulaire interne de la de texte 

#making sequences:
X = tknzr.texts_to_sequences(data['text']) #convertir les mots en entier (sequence de nbr)
X = pad_sequences(X, padding='post', value=0) #assurer que tous les entite en la meme longeur

## Create Numpy Arrays

In [ ]:
X = np.array(X)
y = np.array(data['polarity'])

print(X.shape) #return (nb ligne,nb colone)

(22366, 503)


## Create the model

In [ ]:
model = Sequential() #creer un sequence 
model.add(Embedding(len(tknzr.word_index), 32))
model.add(Bidirectional(GRU(units = 32)))
model.add(Dense(32, activation = 'tanh'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          1926528   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12672     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,941,313
Trainable params: 1,941,313
Non-trainable params: 0
______________________________________________

In [ ]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state=42)


In [ ]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22366 entries, 0 to 22365
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      22366 non-null  object
 1   polarity  22365 non-null  object
dtypes: object(2)
memory usage: 349.6+ KB


In [ ]:
data.dtypes

text        object
polarity    object
dtype: object

In [ ]:
#
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
y=lb.fit_transform(y)

## Train the model

In [ ]:
model.fit(X, y, validation_split = 0.33, epochs = 7, batch_size= 128, shuffle = True)

Epoch 1/7
158/158 [==============================] - ETA: 0s - loss: -0.4295 - accuracy: 0.5387

InvalidArgumentError: ignored

## Tests

In [ ]:
class_names = ['سلبي' , 'إيجابي']
def classify(sentence):
  sentence = process_review(sentence)
  sequence = [tknzr.word_index[word] for word in sentence.split(' ')]
  sequence = pad_sequences([sequence], maxlen = X.shape[1], padding='post', value=0)
  #print(sequence.dtype)
  #print(sequence)
  pred = model.predict(sequence)[0][0]
  print(class_names[np.round(pred).astype('int')], pred)
  

In [ ]:
classify(" و أفكاره معروفة بوسطيتها واعتدالها و قربها من العقل والمنطق ")

إيجابي 1.0


In [ ]:
classify("وحتى بالعدوان كانوا شركاء مجزرة مروحين")

سلبي 1.3441268e-10


In [ ]:
classify(" بتمنى كون صديقة بيناتكن احلى منتدى")

إيجابي 1.0


In [ ]:
classify("سيء")

إيجابي 1.0


In [ ]:
classify("كمن سنة ولما توافدو اليهود مناطق العالم")

سلبي 1.0990065e-14


In [ ]:
classify(" ولما توافدو اليهود مناطق العالم")

سلبي 2.6769916e-11


In [ ]:
import csv
def create_csv(file, dict):
    with open(file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for key in dict.keys():
            writer.writerow([key,dict[key]])

In [ ]:
create_csv("word2index.csv", tknzr.word_index)

In [ ]:
model.save("keras.h5")